In [1]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from kobert_transformers import get_kobert_model, get_tokenizer

In [2]:
MODEL_NAME = "monologg/kobert"
#electra = "monologg/koelectra-small-v2-discriminator"

model = BertForSequenceClassification.from_pretrained(MODEL_NAME)
model.classifier.out_features=1
#model.cuda()
tokenizer = get_tokenizer()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
tokens = tokenizer.tokenize("한국어 모델을 공유합니다")
print("Tokens: {}".format(tokens))

# This is not sufficient for the model, as it requires integers as input, 
# not a problem, let's convert tokens to ids.
tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
print("Tokens id: {}".format(tokens_ids))

# Add the required special tokens
tokens_ids = tokenizer.build_inputs_with_special_tokens(tokens_ids)

# We need to convert to a Deep Learning framework specific format, let's use PyTorch for now.
tokens_pt = torch.tensor([tokens_ids])
print("Tokens PyTorch: {}".format(tokens_pt))

# Now we're ready to go through BERT with out input
output = model(tokens_pt)
#outputs = output['last_hidden_state']
#pooled = output['pooler_output']
#print("Token wise output: {}, Pooled output: {}".format(outputs.shape, pooled.shape))

Tokens: ['▁한국', '어', '▁모델', '을', '▁공유', '합니다']
Tokens id: [4958, 6855, 2046, 7088, 1050, 7843]
Tokens PyTorch: tensor([[   2, 4958, 6855, 2046, 7088, 1050, 7843,    3]])


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

dataset = pd.read_csv('./data/news_train.csv')
train, val, train_label, val_label = train_test_split(dataset['content'], dataset['info'])

In [5]:
train_encodings = tokenizer(list(train.values), truncation=True, padding=True)
val_encodings = tokenizer(list(val.values), truncation=True, padding=True)

#print(tokenizer.convert_ids_to_tokens(train_encodings['input_ids'][0]))

In [7]:
import torch
from torch.utils.data import Dataset

class NewsDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels.values if labels is not None else None
    
    def __len__(self):
        return len(self.encodings[list(self.encodings.keys())[0]])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = NewsDataset(train_encodings, train_label)
val_dataset = NewsDataset(val_encodings, val_label)

In [8]:
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    pbar = tqdm(train_loader, desc=f'Epoch {epoch}, Loss : ')
    for batch in pbar:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        pbar.set_description(f"Epoch {epoch}, Loss : {loss:.3f}")
        optim.step()
        
model.eval()

Epoch 0, Loss : 0.407:   0%|          | 8/2784 [00:09<52:04,  1.13s/it]


KeyboardInterrupt: 

In [26]:
test = pd.read_csv('./data/news_test.csv')
test_encodings = tokenizer(list(test['content'].values), truncation=True, padding=True)
test_dataset = NewsDataset(test_encodings)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False, num_workers=4)
info_list = []
model.eval()
for batch in tqdm(test_loader):
    with torch.no_grad():
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        info_list.append(torch.max(outputs[0], dim=1)[1])
info_list = torch.cat(info_list)

Exception ignored in: <function tqdm.__del__ at 0x7f124c5ce940>
Traceback (most recent call last):
  File "/mnt/serverhdd2/jiwook/anaconda3/envs/nh/lib/python3.8/site-packages/tqdm/std.py", line 1123, in __del__
    def __del__(self):
KeyboardInterrupt: 


In [22]:
submission = pd.read_csv('./data/sample_submission.csv')

In [24]:
submission['info'] = info_list

ValueError: Length of values (27) does not match length of index (142565)